# Markov Models

This notebook explores how to implement Markov models using basic Python syntax, starting with simplest possible form of this problem.

In [1]:
from collections import defaultdict
from random import randint

## Helpful Python Syntax
#### Skip this section if you have a solid Python background

The zip command allows for simultaneous interation through multiple objects.  The most typical use case is to loop through the contents of two lists at the same time.  Upon each step of the zip iteration, a tuple is returned.  This can be unpacked immediately (see the use of "i, j" below) or the entire tuple stored.

In [2]:
a = [1,2,3,4]
b = [4,5,9,10]

print('unpack as we iterate')
for i, j in zip(a,b):
    print(i,j)
    
print("\njust iterate")
for values in zip(a,b):
    print(values)

unpack as we iterate
1 4
2 5
3 9
4 10

just iterate
(1, 4)
(2, 5)
(3, 9)
(4, 10)


The zip command provides a handy way to iterate through a single object while grabbing multiple items.  This can be done by using the same input list for multiple argurments and slicing to trim them down.

(Technical note: Your input object in this case must be an iterable that does not exhaust or you need to make a copy of it.  No worries for lists.)

(Technical note 2: "Better" solutions for iterating through pairs of items exist.  Read more about itertools or better yet how generators work if you want a more robust formulation.)

In [3]:
a = [1,2,3,4]
b = ['red','blue','green','purple']

b[:-1]


['red', 'blue', 'green']

In [4]:
print(b[:-1])
print(b[1:])
print()
for prev, cur in zip(b[:-1], b[1:]):
    print(prev, cur)

['red', 'blue', 'green']
['blue', 'green', 'purple']

red blue
blue green
green purple


Next we need to talk a little about details of the python dictionary.  A dictionary creates a mapping between pairs of keys and values.  Something new Python users don't immediately realize is that keys can be any immutable object.  In practical terms, this means that tuples are an excellent choice.  Additionally, the values themselves can be an arbitrariy complext object, such as another dictionary.  This enables more involved mappings:

In [5]:
simple_dict_0_0 = {1: 'a', 2: 'b'}
simple_dict_1_0 = {'baseball': 'bat', 'tennis': 'racket'}
simple_dict_0_1 = {}
simple_dict_1_1 = {'solution to everything': 42}

print(simple_dict_0_0[1])
print(simple_dict_1_0['baseball'])
print(simple_dict_1_1)

a
bat
{'solution to everything': 42}


In [6]:
my_dict = {}
my_dict[(0,0)] = simple_dict_0_0
my_dict[(0,1)] = simple_dict_0_1
my_dict[(1,0)] = simple_dict_1_0
my_dict[(1,1)] = simple_dict_1_1

my_dict

{(0, 0): {1: 'a', 2: 'b'},
 (0, 1): {},
 (1, 0): {'baseball': 'bat', 'tennis': 'racket'},
 (1, 1): {'solution to everything': 42}}

In [7]:
print('Access a sub-dictionary for my_dict using []')
print(my_dict[(1,1)])

print('\nAccess a single value of taht sub-dictionary for my_dict using [] twice in a row')
print(my_dict[(1,1)]['solution to everything'])

Access a sub-dictionary for my_dict using []
{'solution to everything': 42}

Access a single value of taht sub-dictionary for my_dict using [] twice in a row
42


Note in the cells above that we create a dictionary whose values are other dictionaries.  In this case, looking up a key with [ ] syntax returns another dictionary, in which another lookup using [ ] can be used immedately.  An example is shown in the last line of the cell above.

## Building a Markov Model

Consider a system which is described at each instance by a state, and this state transitions to different states.  A classic example is the weather.  The current state could be described by temperature, windspeed, humity, and a number of other parameters.  Tomorrow, the system will have a different state.

A Markov process in one in which the transition probabability of going to the next state is assumed to be entirely described by information in the current state.  Consider predicting tomorrow's weather.  If we model it as a Markov process, we are saying that information about today's weather will drive our prediction, but information about the yesterday's weather provides no additional information.

We could apply such a model to describe English sentances, where the probability of a word appearing is assumed to be only a function of the word before it.  This is a very crude approximation for how English syntax, but it gives us a starting place for a simple model.

Text processing is a rich field unto itself, but for moment we want to focus on the mechanics of Markov models.  As such, will will start with a simple copy-paste of some text (in this case from the Wikipedia page on fire), and do minimal pre-processing.  The result is a list of words, where '.' is treated as a word.  Effectively we will be using '.' as SOS and EOS token, but let's gloss over those details for now.

In [8]:
raw_text = "Fire is the rapid oxidation of a material in the exothermic chemical process of combustion, releasing heat, light, and various reaction products.[1] Slower oxidative processes like rusting or digestion are not included by this definition.  Fire is hot because the conversion of the weak double bond in molecular oxygen, O2, to the stronger bonds in the combustion products carbon dioxide and water releases energy (418 kJ per 32 g of O2); the bond energies of the fuel play only a minor role here.[2] At a certain point in the combustion reaction, called the ignition point, flames are produced. The flame is the visible portion of the fire. Flames consist primarily of carbon dioxide, water vapor, oxygen and nitrogen. If hot enough, the gases may become ionized to produce plasma.[3] Depending on the substances alight, and any impurities outside, the color of the flame and the fire's intensity will be different. Fire in its most common form can result in conflagration, which has the potential to cause physical damage through burning. Fire is an important process that affects ecological systems around the globe. The positive effects of fire include stimulating growth and maintaining various ecological systems. The negative effects of fire include hazard to life and property, atmospheric pollution, and water contamination.[4] If fire removes protective vegetation, heavy rainfall may lead to an increase in soil erosion by water.[5] Also, when vegetation is burned, the nitrogen it contains is released into the atmosphere, unlike elements such as potassium and phosphorus which remain in the ash and are quickly recycled into the soil. This loss of nitrogen caused by a fire produces a long-term reduction in the fertility of the soil, which only slowly recovers as nitrogen is fixed from the atmosphere by lightning and by leguminous plants such as clover. Fire has been used by humans in rituals, in agriculture for clearing land, for cooking, generating heat and light, for signaling, propulsion purposes, smelting, forging, incineration of waste, cremation, and as a weapon or mode of destruction."

In [9]:
text = ['.'] + raw_text.lower().replace(".", " . ").split()

Before we start building the model, it is informative to consider the simplier task counting how many times each word appears.  Below are two cells each performing the same task.  The first uses a dictionary and the second uses a defaultdict.

While both datastructures give the same result, the latter has the advantage of assuming any new key maps to a value of zero.  This elimnates the need to check if we have already encountered a key.  As we will see, default dict also makes it convenient to built more complicated structures.

In [10]:
word_count = {}
for word in text:
    if word not in word_count.keys():
        word_count[word] = 1
    else:
        word_count[word] += 1 # same as: word_count[word] = word_count[word] + 1 

print("How man times does the word 'the' appear?")        
print(word_count['the'])

How man times does the word 'the' appear?
29


In [11]:
word_count = defaultdict(int)
for word in text:
    word_count[word] += 1

print("How man times does the word 'the' appear?")     
print(word_count['the'])

How man times does the word 'the' appear?
29


The Markov Model needs to represent the possiblity of each word transitioning to another word.  One could imagine this as a large matrix of probabilities, however, most word-word transitions never occur (at least not in our training input).  For example, you do not expect to see an adjective followed by an article such as 'chemical' -> 'the'.  Rather than use a matrix of mostly zeros, we are going to use a defaultdict to store possible transitions.

(Technical note: scipy.sparse provides an implementation of sparse matricies which would an excellent storage choice for this problem.  This problem could also be seen as a graph creation and other packages such as networkx would be excellent way to convert word pairs into a directed graph.  However, this notebook is focused using basic python functionality.)

The transitions will be stored in a defaultdict, where each key is a single word and the each value is a list of words come after it.  This is NOT the best structure for the final model, but is a convenient choice for a short input text where we want to see some of the inner workings of the process.  We will switch to a better representation later on.

For tracking the possible next words, the default value in our dictionary will be a list.  Whereas for counting we used '+= 1' to increment an int, this time we append to a list of words that have been observed to come after the left word.

In [12]:
observed_trans = defaultdict(list)
for left, right in zip(text[:-1], text[1:]):
    observed_trans[left].append(right)

print(observed_trans['fire'])

['is', 'is', '.', 'in', 'is', 'include', 'include', 'removes', 'produces', 'has']


The obsereved_tran['fire'] tells us which words came after 'fire' in our training text.  The pair "fire is" occured multiple times in the text, so 'is' is listed multiple times in the list.

In [13]:
print(observed_trans['.'])

['fire', '[1]', 'fire', '[2]', 'the', 'flames', 'if', '[3]', 'fire', 'fire', 'the', 'the', '[4]', '[5]', 'this', 'fire']


Note that '.' is sometimes followed by words like '[1]' because we did nothing to clean up citation references in the original text.

Now that we know what transitions occur, and how often, we can start doing some interesting things like sampling the distrubution to generate new sentances.  This is done by picking '.' as our starting word, randomly choosing a word that could follow it, printing that word, and then restarting the process with the new current word.  We stop when we enconter another period.

Try running the following cell multiple times to see what all outputs it produces.

In [14]:
cur = '.'

for _ in range(30):
    possible = observed_trans[cur]
    cur = possible[randint(0,len(possible)-1)]
    print(cur)
    if cur=='.':
        break
    

this
loss
of
nitrogen
it
contains
is
burned,
the
bond
energies
of
combustion,
releasing
heat,
light,
and
water
contamination
.


The output is mostly a strangly meandering 'sentence' that almost makes sence at times, but is not quite right.  This is because the core assumption of the model (that the probaility of each word occuring is solely a function of the previous word) is flawed.  We will work to show how we can improve this model with additional complexity.  

However, even though this model not sufficient to generate good new language, it has some potential use.  For example, it can tell us how reasonable an existing sentance is.  In the simplest form, we could see how likely a new sentance is.

In [15]:
raw_text_a = "fire is the fire."
raw_text_b = "The the the the."

# We have to apply the same text preprocessing to test text as we did our original training data
test_text_a = ['.'] + raw_text_a.lower().replace(".", " . ").split()
test_text_b = ['.'] + raw_text_b.lower().replace(".", " . ").split()

In [16]:
# test a
cum_prob = 1
for left, right in zip(test_text_a[:-1], test_text_a[1:]):
    prob = observed_trans[left].count(right) / len(observed_trans[left])
    cum_prob *= prob
    print("Transition: '{}' to '{}' has observed probability {}".format(left, right, prob))
print("Total prob of sentance:", cum_prob)

Transition: '.' to 'fire' has observed probability 0.3125
Transition: 'fire' to 'is' has observed probability 0.3
Transition: 'is' to 'the' has observed probability 0.2857142857142857
Transition: 'the' to 'fire' has observed probability 0.034482758620689655
Transition: 'fire' to '.' has observed probability 0.1
Total prob of sentance: 9.236453201970444e-05


In [17]:
# test b
cum_prob = 1
for left, right in zip(test_text_b[:-1], test_text_b[1:]):
    prob = observed_trans[left].count(right) / len(observed_trans[left])
    cum_prob *= prob
    print("Transition: '{}' to '{}' has observed probability {}".format(left, right, prob))
print("Total prob of sentance:", cum_prob)

Transition: '.' to 'the' has observed probability 0.1875
Transition: 'the' to 'the' has observed probability 0.0
Transition: 'the' to 'the' has observed probability 0.0
Transition: 'the' to 'the' has observed probability 0.0
Transition: 'the' to '.' has observed probability 0.0
Total prob of sentance: 0.0


In these examples, the probability of transition was assumed to be the same a the fraction of time the same transition was observed in training data.  For example 'is' occured 3 times after the word 'fire' out of the time times 'fire' existed in the training text, so the probability was 0.3 or 30%.  The probability of an entire sentance was then assumed to be the product of all the individual word transitions.

This is roughly correct, but has a serious shortcomming.  It assumes that anything transition not observed is impossible.  For this reason, a good model needs to use smoothing (or an alternative technique) to deal with the fact that the training set is not a complete description of all possible transitions.

## Conclusion

From here, there are several pathways to explore for improving the model.  First, smoothing is vital to deal with overfitting by the model.  Second, this was a true "Markov Model", rather than a "Hidden Markov Model", so extending the model to have both transition and emission probabilities is necessary to create a more robust tool.  Third, our model only used the previous word for transitions, not the previous two words, or three.  Extending the definition of the state can allow the model to more realistically mimic English syntax.  However, techniques like smoothing become even more vital or the model will just overfit the training data more.

Beyond the model itself, the way we used it to describe and generate new sentances was crude.  Once a better model is created based on the methods of the previous paragraph, Vitterbi and other algorithms provide smarter ways of leveraging the model.

This notebook provided basic overview and samples of using core Python tools to dig into a real Machine Learning problem, and providing an hands-on sample of what Markov processes are about.  A future notebook will explore the NLP problem more deeply.